# Tutorial 11

Task 1 : Make a class for a filter read from a FITS file. Each object of this class should be for a specific filter(for instance, U/G/R/I/Z). The class should have the following attributes:

1. An array containing the quantum efficiency values for that filter as an instance attribute.
2. An array containing the wavelength values as an instance attribute.


The class should contain the following functions :

1. An __init__() function which initializes the array for quantum efficiency .
2. A member function calc_intensity() which calculates the intensity of a star given an object of a star class as its argument.

You can use the code that you wrote in Tutorial 10 in this task directly.

In [1]:
import numpy as np
from astropy import units as u
from astropy.io import fits
from astropy import constants as const

In [2]:
h = const.h
c = const.c
kB = const.k_B

def B(wl,T):
    exponential = 1/(np.exp(h*c/(wl*kB*T))-1)
    prefactor = 2*np.pi*h*c**2/wl**5
    return prefactor*exponential

In [3]:
class Filter(object):
    hdul = fits.open('filter_curves.fits') # Class attribute (required for every object of the class)
    
    def __init__(self, name: str):                 # Instance attribute
        hdu = self.hdul[name]
        data = hdu.data
        self.name = name
        self.q = data['respt']
        self.wl = data['wavelength']*u.AA

    def print_filter(self):
        print(self.q)
        print(self.wl)

class star(Filter):
    
    def __init__(self, name: str, temp, rad, dist):
        super().__init__(name)
        self.T = temp*u.K
        self.R  = rad*u.Rsun
        self.D = dist.to(u.lightyear)

    def print_star(self):
        print(f'Temperature={self.T}  \n Radius={self.R} \n Distance={self.D}')

    def calc_intensity(self):
        integral = 0*u.W/u.m**2
        i = 1
        while i < len(self.wl):
            integral += B((self.wl[i-1]+self.wl[i])/2 , self.T)*(self.q[i-1]+self.q[i])/2*(self.wl[i]-self.wl[i-1])
            i = i + 1
        intensity = integral*(self.R/self.D)**2
        
        return intensity.to(u.W/u.m**2)

In [4]:
star_G = star('G', 9940, 1.711, 2.64*u.pc) # Distance is given in parsec
print(star_G.print_star())

Temperature=9940.0 K  
 Radius=1.711 solRad 
 Distance=8.610528371722024 lyr
None


In [5]:
print(star_G.calc_intensity())

7.024221695949648e-09 W / m2
